<p style="page-break-after:always;"></p>

# Forrestania (Joint Grav/Mag/EM)

This uses public data from the Geological Survey of Western Australia over the Forrestania geological province.

<p style="page-break-after:always;"></p>

## Data

We first need to consolidate the data into a usable format. We use [Geoscience ANALYST](https://www.mirageoscience.com/mining-industry-software/geoscience-analyst/) importer to quickly convert the original file formats into `geoh5` with a simple `drag & drop` to the viewport. 
The original datasets comprise

 - Airborne magnetic survey: `60472_AOI4.gdb`
 - Ground gravity survey: `Forrestania_Gravity_Station_trim_.csv`
 - Digital Elevation Model (DEM): `Forrestania_SRTM1 Australia_MGA50.ers`

```{figure} ./images/data_import.png
---
name: data_import

---
Data import from their original format to `geoh5`.
```

Once imported, the data can be activated for visualization.  

### Processing

Note that both the DEM and airborne magnetic survey were imported at an arbitrary 0 m elevation and do not reflect the true position.
We first need to modify our object such that the data is accurately located in 3D space. 

#### Step 1: Convert the 2D DEM into a 3D surface
```{figure} ./images/create_surface.png
```

#### Step 2a: Transfer the DEM data onto the airborne survey (curve). 
```{figure} ./images/transfer_dem.png
```

#### Step 2b: Re-assign the Z channel to the DEM + radar height.
```{figure} ./images/drape_mag.png
```



## Gravity

We begin our analysis with the ground gravity survey. We will attempt to estimate the shape and depth of the causative body at depth.

### Inputs

Gravity data are sensitive to variations in the density of rocks. The data were already processed and provided as terrain-corrected (2.67 g/cc) residual data. We can therefore go directly to the inversion phase. 

```{figure} ./images/gravity_data.png
```

### Inversion options

The following options are set for the inversion.

```{figure} ./images/gravity_setup.png
```
- Assign the data uncertainty to a quarter of the standard deviation (0.05 mGal). This is a somewhat conservative estimate that takes into account both the instrumental and numerical noise.
- Use a `0 g/cc` reference value for the model. This assumes that the signal from the background density was accurately subtracted from the data.
- Leave the `Mesh` option unchecked. We will let the code create an octree mesh based on the average data separation and extent.

### Inversion results

After completion of the inversion, we load the results and assess convergence and data fit.

```{figure} ./images/grav_predicted.png
```

We note the following:

 - It took 13 iterations to converge to the target misfit.
 - The following 12 iterations have increased the model complexity ($phi_m$) but preserved the data fit ($phi_d$).
 - Most of the important gravity anomalies appear to be reproduced by the final model.

#### Smooth solution (L2-norm)

```{figure} ./images/grav_l2_model.png
```

The smooth solution (iteration 13) indicates the presence of a dense anomaly extending at depth with density contrasts ranging from [-0.02, 0.05] g/cc. The negative density contrasts appear to be localized around the main positive anomaly, likely due to the smoothness constraint and the lack of `bound` constraints. Some smaller "fuzzy" anomalies are visible at the margins.  

#### Compact solution (L0-norm)

```{figure} ./images/grav_l2_model.png
```

As an alternative solution, the compact model (iteration 25) recovers a well-defined dense body within a mostly uniform background. Density contrasts have substantially increased in the range of [0.0, 0.56] g/cc.

<p style="page-break-after:always;"></p>

## Magnetics

We follow up with the processing of the airborne magnetic data.

### Inputs

The source of the magnetic signal can generally be attributed to the presence or destruction of magnetic minerals in rocks (mainly magnetite). Magnetometers measure the Total Magnetic Intensity (TMI) that includes both the primary (source) field and secondary fields (signal) from the local geology. The first step is to remove the primary (inducing) field at the time and location of acquisition.

#### Step 1: Set the coordinate system of survey
```{figure} ./images/mag_coordinates.png
```

#### Step 2: Lookup/remove IGRF 


#### Step 3: Downsampling

We can downsample along line to reduce the computation cost and bring the resolution closer to the gravity survey.

```{figure} ./images/mag_downsample.png
```

### Inversion options

The following options are set for the inversion.

```{figure} ./images/gravity_setup.png
```